In [1]:
import numpy as np

In [2]:
workdir = "/Volumes/Spare Data/Hannah_Data/"
inner_work = 'hor1flcs/'
seFD = "seFLCs/"
extF = "sortedFLCs.dat"
saveDir = "catRawMags1703/"

In [3]:
EEVband = 0.795 #3 pix rad
EEIband  = 0.770 #3 pix rad
# EEVband = 0.656 # 2 pix rad
# EEIband = 0.603 # 2 pix rad
# EEVband = 0.313 # 1 pix rad
# EEIband = 0.293 # 1 pix rad

ZPV = 26.667
ZPI = 26.779

In [5]:
# targnamesList = np.loadtxt("targnamesDirections.txt",dtype=str)
filterList = ["F606W","F814W"]

In [6]:
def getJdan(targname,filt):

    jdanList_file = np.loadtxt(workdir+targname+"_flcs.txt",dtype="<U50")

    jdan = jdanList_file[:,0]
    jdanFilter = jdanList_file[:,1]

    jdanUse = []
    for jj in range(len(jdan)):
        if jdanFilter[jj]==filt:
            jdanUse.append(jdan[jj])

    jdanUse = np.array(jdanUse)

    return jdanUse

In [12]:
def get_mags(targname,filt):
    jdanUse = getJdan(targname,filt)

    for ff in range(len(jdanUse)):
        #Load sextractor file catalog
        sex_f = workdir+inner_work+seFD+"se_"+jdanUse[ff]+'_'+targname+'_'+filt+'.dat'
        
        # x,y,flux,class_star
        cat = np.loadtxt(sex_f,comments='#',dtype=None,usecols=(3,4,5,14))
        flux = cat[:,2]
        exptime = get_expt(jdanUse[ff])
        
        newCol = np.zeros((len(cat),2))
        
        newCol[:,0] = flux2mag(flux,filt,exptime)
        newCol[:,1] = np.arange(0,len(cat),1,dtype=int)
        
        cat = np.hstack((cat,newCol))
        
        header = "xr yr flux c_star magr id"
        
        np.savetxt(workdir+inner_work+saveDir+jdanUse[ff]+'_'+targname+'_'+filt+'_'+'wMag.dat',
                  cat,fmt="%1.5f %1.5f %1.5f %1.4f %1.5f %d",header=header)
        
    return None

In [8]:
def get_expt(jdan):
    file = np.loadtxt(workdir+extF,usecols=(0,4),
                         comments='#',dtype=str)
    targname = jdan+'_WJC.fits'
    exptime = file[:,1][file[:,0]==targname]
#     exptime = file['EXPTIME'][file['flcName']==targname]
    
    return float(exptime)

In [9]:
def flux2mag(flux_arr,filt,exptime):
    if filt=='F606W':
        EE = EEVband
        ZP = ZPV
    else:
        EE = EEIband
        ZP = ZPI
        
    mags = -2.5*np.log10(flux_arr/EE) + 2.5*np.log10(exptime) + ZP
    
    return mags

In [13]:
targname = 'HOROLOGIUM-I'
for ff in range(len(filterList)):
    get_mags(targname,filterList[ff])

/Users/hr8jz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in log10
  if __name__ == '__main__':
